In [1]:
file = open('data/retail.dat')
file_lines = file.readlines()
transactions = []
support = 20
confidence = .2
C = {}
for i in file_lines:
    transactions.append(i.strip().split())
    if len(transactions) == 1000:
        break
items = set()
for i in range(len(transactions)):
    for j in range(len(transactions[i])):
        items.add(frozenset([transactions[i][j]]))
for it in items:
    for i in range(len(transactions)):
        if it.issubset(transactions[i]):
            if it in C:
                C[it] += 1
            else:
                C[it] = 1
L = {}
for i,j in C.items():
    if j >= support:
        L[i] = j

In [2]:

def getL(support, items, lent, transactions):
    items2 = list(items.keys())
    L_set = set()
    for i in items2:
        for j in items2:
            if len(i.union(j)) == lent:
                L_set.add(i.union(j))
    C = {}
    for i in range(len(transactions)):
        for it in L_set:
            if it.issubset(transactions[i]):
                if it in C:
                    C[it] += 1
                else:
                    C[it] = 1
    L = {}
    for i, j in C.items():
        if j >= support:
            L[i] = j
    return L

In [3]:
lent = 2
TL = {}
TL[1] = L
while True:
    new_L = getL(support, L, lent, transactions)
    if len(new_L) == 0:
        break
    TL[lent] = new_L
    L = new_L
    lent += 1
for i,j in L.items():
    print(i, ' ', j, '\n')

frozenset({'39', '38', '41', '48'})   41 



In [4]:
import itertools

result = {}
for it in L:
    ele = it
    for i in range(1, len(it)):
        for a_set in itertools.combinations(it, i):
            b_set = it - frozenset(a_set)
            a_set = frozenset(a_set)
            con = TL[len(a_set.union(b_set))][a_set.union(b_set)] / TL[len(a_set)][a_set]
            if con >= confidence:
                result[str(set(a_set)) + '->' + str(set(b_set))] = con
                
for i,j in result.items():
    print(i, ' ', j, '\n')

{'38', '39'}->{'41', '48'}   0.2662337662337662 

{'41', '39'}->{'38', '48'}   0.2192513368983957 

{'38', '41'}->{'39', '48'}   0.47674418604651164 

{'38', '48'}->{'41', '39'}   0.36936936936936937 

{'41', '48'}->{'38', '39'}   0.3178294573643411 

{'38', '41', '39'}->{'48'}   0.6119402985074627 

{'38', '39', '48'}->{'41'}   0.5 

{'41', '39', '48'}->{'38'}   0.3867924528301887 

{'38', '41', '48'}->{'39'}   0.8541666666666666 

